# 自动出图

流程：
- 让ai帮我们生成prompt，然后保存到某个txt文件里
- 以这个txt文件为输入，调用已经写好的ComfyUI脚本，生成图片

In [1]:
# 先得安装pyautogen
%pip install pyautogen

In [2]:
# 设置Api入口
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [3]:
# 创建完成这个项目需要的agent
llm_config = {
    "config_list": config_list, 
    "seed": 42,
    "functions": [
        {
            "name":"saveFile",
            "description":"save the generated text to a file",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "the message to be saved",
                    },
                },
                "required": ["message"],
            },
        }
    ],
    }

# 首先，我们需要一个与人类用户交互的proxy agent
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE",
)
# 接下来，我们需要一个把用于中英文翻译的agent
translator = autogen.AssistantAgent(
    name="Translator",
    system_message="Your task is to translate the user's Chinese into English, and translate the English messages returned by other agents into Chinese for display to the user.",
    llm_config=llm_config
)
# 然后，我们需要一个把按用户需求生成绘图prompt的agent
designer = autogen.AssistantAgent(
    name="designer",
    system_message="""
    Please generate 30 image descriptions that meet the user's needs:
    - Include description of attire, background, lighting and shadow.
    - Each sentence should be limited to 50 words.
    """,
    llm_config=llm_config
)

# # 我们还需要一个能把prompt文件保存为text的agent
# saver = autogen.UserProxyAgent(
#     name="FileSaver",
#     code_execution_config={"last_n_messages": 2, "work_dir": "output"},
#     human_input_mode="NEVER",
#     is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
#     max_consecutive_auto_reply=10,
# )

def saveFile(message):
    print("got message: ", message)
    save_file_path = "output\generated_text.txt"
    try:
        with open(save_file_path, "a") as f:
            f.write(message["content"] + "\n")
        return f"image prompt file generated ,and saved at {save_file_path}"
    except Exception as e:
        print(e)
        return f"image prompt file generated failed, error: {e}"

user_proxy.register_function(
    function_map={
        "saveFile": saveFile,
    }
)

# 然后，我们建一个群把它们放在一起
groupchat = autogen.GroupChat(agents=[user_proxy, translator, designer], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [4]:
# 用户输入需求
user_input = "帮我生成一些女网红的写真照片的提示词，然后保存成文件。"

In [5]:
# 执行代码
user_proxy.initiate_chat(manager,message=user_input)

User_proxy (to chat_manager):

帮我生成一些女网红的写真照片的提示词，然后保存成文件。

--------------------------------------------------------------------------------
Translator (to chat_manager):

Okay, let's first create the prompt sentences for the female internet celebrity photoshoot. 

1. Show your elegance in a beautiful dress with your face in profile.
2. Capture the moment when you are concentrating on reading a book.
3. Show your playful personality by blowing a bubble with a gum.
4. Capture a casual moment in your pajamas and a messy bun.
5. Highlight your athleticism with a shot of you playing any sport.
6. Take a picture of you experimenting with cooking in the kitchen.
7. A close-up of you sipping a coffee in a cute cafe will give a lifestyle vibe.
8. A candid shot of you laughing with your friends can show your cheerful nature.
9. Reflect your adventurous side with a shot of you hiking or doing an outdoor activity.
10. Show your professional side with a shot in your work environment.

Now I will s